# Authentication and authorization
This demo will show following actions:
- login creation
- server role creation
- database user creation
- permissions hierarchy
- setting permissions for principals
- observing effective permissions for some user
- moving SQL logins between instances of SQL Server

## How to create logins
Following statements will create Windows login and SQL login. We can see couple of interesting properties configured in SQL login creation:
- *CHECK_POLICY*: SQL Server uses GPO to check for password policies. For legacy systems or for users who are not allowed to change password on regular basis it is needed to check to password policy off.
- *DEFAULT_DATABASE*: The authentication is againts an instance and a database. The database must exist! If it does not, the authentication fails. Database master has **guest** database user account always enabled, so the default_database whould be left on its default value - master.
- *DEFALUT_LANGUAGE*: It is a language of error messages, not a content of user data! If you want to make some canadian joke, switch it to something actually exotic.

In [ ]:
use master
go

-- windows login
if exists(select * from sys.server_principals where name = 'adventureworks\marek')
    drop login [adventureworks\marek]
go

-- create login [adventureworks\marek] from windows
-- go

-- sql login
if exists(select * from sys.server_principals where name = 'david')
    drop login David

create login David with password = 'Pa$$w0rd', check_policy = off, default_database = master, default_language = English
go

## Server role creation

Since 2012, SQL Server allows to create server-level user roles. It is useful when some SQL Server users need to have some server level privileges, i. e. a creation of new databases.

In [ ]:
use master
go

if exists(select * from sys.server_principals where type = 'R' and name = 'nonAdmins')
    drop server role nonAdmins
go

create server role nonAdmins
go

alter server role nonAdmins add member David
go


## Creating database user
Very simple statement, let's execute it. When the user is being created, we can set its *DEFAULT_SCHEMA*. It allows the user to ommit prefix of the schema in statements. Avoid this technique, it leads to:
1. Schema name resolution (performance impact)
2. Unstable, not reliable code (the property can be changed any time)

In [ ]:
use Demo
GO

if exists(select * from sys.database_principals where name = 'David')
    drop user David;
go

create user David for login David with default_schema = dbo
go

## Working with permissions
Principals, when created, have very short list of permissions. Usually we need to set more permissions to principals. First of all, it's good to know how to ask for simple "cheat sheet" showing a hierarchy of permissions. Let's execute following query.

In [ ]:
declare @level nvarchar(25) = 'SERVER'
;with cte as
(
select permission_name, covering_permission_name, 1 as permission_level from sys.fn_builtin_permissions(@level) where covering_permission_name = ''
union all
select p.permission_name, p.covering_permission_name, cte.permission_level + 1 from sys.fn_builtin_permissions(@level) as p join cte on p.covering_permission_name = cte.permission_name
)
select * from cte

Preceding query can take also other parameter's values: **DATABASE** and **SCHEMA**.

Now we can assign some permissions to, say, database user David.

In [ ]:
GRANT INSERT, SELECT, UPDATE ON OBJECT::TestTable TO David

## Observing current permissions assigned to user

Even if we have an option to right-click each user or object using _SSMS_ and observe assign properties using the GUI approach, sometimes it's very feasible to have bigger picture of permissions assigned to user. Following query uses some metadata objects to show who has which permissions.

In [ ]:
use Demo
GO

select
	dp.name as user_name
	, p.class_desc
	, p.permission_name
	, s.name as schema_name
	, concat(OBJECT_SCHEMA_NAME(o.object_id), '.', o.name) as full_object_name
	--, p.*
from sys.database_principals as dp
	join sys.database_permissions as p on dp.principal_id = p.grantee_principal_id
	left join sys.schemas as s on p.major_id = s.schema_id and p.class_desc = 'SCHEMA'
	left join sys.all_objects as o on p.major_id = o.object_id and p.class_desc = 'OBJECT_OR_COLUMN'
where dp.name = 'David'
	and p.state_desc = 'GRANT'
	and p.class_desc != 'DATABASE'

## Moving SQL logins between instances of a SQL Server
Many times it happens that a database is migrated between instances of SQL Server. It's not an issue for Windows login when the database is being moved within a domain, because the login's SID remains the same. But we should migrate SQL logns as well, and we need to do it with following query. The query extracts all SQL Logins and keeps their SIDs and hashed passwords.

In [ ]:
;with logins as
(
select name
, convert(varchar(max), cast(sid as varbinary(max)), 1) as sidString
, CONVERT(varchar(max), cast(LOGINPROPERTY(name, 'PasswordHash') as varbinary(max)), 1) as PwdHashString
from sys.server_principals where type = 'S'
and not name = 'sa' and not name like '##%'
)
select
concat('CREATE LOGIN ', name, ' with password = ', PwdHashString, ' hashed, sid = ', sidString) as sql_statement
from logins